In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_excel('/home/jefferyp/Documents/projects/Cashflow_Forecasting/selectionscashflows_2024.xlsx')

In [3]:
df_obj = df.select_dtypes('object')

In [4]:
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [5]:
# df2 = df.melt(['Activity', 'Total'], var_name='Date', value_name='Measure')
# df2.loc[df2['Activity']=='Credit Card Receivable'].fillna(0)

In [6]:
df3 = df.melt(['Activity'], var_name='Date', value_name='Measure').fillna(0)
# df3.loc[df3['Activity']=='Credit Card Receivable'].fillna(0)

In [7]:
df3 = df3.drop(df3[df3['Measure'] == 0].index)
# df3

In [8]:
df_totals = df3.groupby(['Activity']).sum('Measure')

In [9]:
df_totals = df_totals.drop(df_totals[df_totals['Measure'] == 0].index)
# df_totals

In [10]:
mask = df3['Measure'] < 0
df3['cash_inflow'] = df3['Measure'].mask(mask)
df3['cash_outflow'] = df3['Measure'].mask(~mask)
df4 = df3.drop(['Measure'], axis=1).fillna(0)

In [11]:
df4['net_cash_flow'] = df4['cash_inflow'] - df4['cash_outflow'].abs()

In [12]:
index_names = df4[df4['Date'] == 'Total'].index 
df4.drop(index_names, inplace = True) 

In [13]:
# df4['date_of_sale'] = datetime.strptime(df4['Date'], '%B %d, %Y').strptime('%Y-%m-%d')
df4['Date'] = list(map(lambda x: datetime.strptime(x,'%b %d, %Y').strftime('%m/%d/%Y'), df4['Date']))

In [14]:
df4['Date'] = pd.to_datetime(df4['Date'] )
# df4

In [15]:
sf = pd.read_csv('/home/jefferyp/Documents/projects/Cashflow_Forecasting/2024_sales_cogs_export.csv')

In [21]:
sf['Date'] = pd.to_datetime(sf['date_of_sale'])
sf = sf.drop(['date_of_sale'], axis=1)

In [22]:
sf.head()

,gross_sales,cogs,Date
0,49.00,18.60,2024-01-01
1,418.18,40.08,2024-01-02
2,779.70,284.25,2024-01-03
3,1231.11,521.58,2024-01-04
4,962.33,305.97,2024-01-05


In [27]:
# df_merge = df4.merge(sf, how='cross')
df_merge = pd.merge(df4, sf, on='Date')

In [34]:
df_merge.head(20)

,Activity,Date,cash_inflow,cash_outflow,net_cash_flow,gross_sales,cogs
0,Net Income,2024-01-01,0.00,-1345.95,-1345.95,49.00,18.60
1,11003 BG Blue Amex - GG,2024-01-01,1136.95,0.00,1136.95,49.00,18.60
2,3079 Chase CC - BGG,2024-01-01,0.00,-39.00,-39.00,49.00,18.60
3,63008 Amex Business Platinum CC - GG,2024-01-01,209.00,0.00,209.00,49.00,18.60
4,Total Adjustments to reconcile Net Income to N...,2024-01-01,1306.95,0.00,1306.95,49.00,18.60
5,Net cash provided by operating activities,2024-01-01,0.00,-39.00,-39.00,49.00,18.60
6,Retained Earnings,2024-01-01,13535.04,0.00,13535.04,49.00,18.60
7,Net cash provided by financing activities,2024-01-01,13535.04,0.00,13535.04,49.00,18.60
8,Net cash increase for period,2024-01-01,13496.04,0.00,13496.04,49.00,18.60
9,Net Income,2024-01-02,0.00,-244.85,-244.85,418.18,40.08


In [46]:
df4_totals = df4.groupby(['Activity']).sum('net_cash_flow')

In [47]:
df4_totals

,cash_inflow,cash_outflow,net_cash_flow
Activity,,,
11003 BG Blue Amex - GG,32643.85,-37844.20,-5200.35
3079 Chase CC - BGG,0.00,-39.00,-39.00
3656 Chase CC - SEL:Chase Ink #4304 (Jeff),922.92,0.00,922.92
3656 Chase CC - SEL:Chase Ink #8502 (Danielle),4589.95,-6471.44,-1881.49
4003 Chase Business Credit Line - SEL,8899.28,-9169.99,-270.71
63008 Amex Business Platinum CC - GG,228878.32,-218121.06,10757.26
7001 Chase Business Credit Line - GG,18716.42,-15607.34,3109.08
73003 Business Platinum CC - SEL,37809.46,-37711.65,97.81
AP Clearing Inventory Purchases,0.00,-19547.78,-19547.78


In [48]:
df_totals = df_merge.groupby(['Activity']).sum('net_cash_flow')

In [49]:
df_totals

,cash_inflow,cash_outflow,net_cash_flow,gross_sales,cogs
Activity,,,,,
11003 BG Blue Amex - GG,32643.85,-37844.20,-5200.35,131715.64,53471.91
3079 Chase CC - BGG,0.00,-39.00,-39.00,49.00,18.60
3656 Chase CC - SEL:Chase Ink #4304 (Jeff),922.92,0.00,922.92,19543.36,8180.85
3656 Chase CC - SEL:Chase Ink #8502 (Danielle),4589.95,-6471.44,-1881.49,35856.34,14506.10
4003 Chase Business Credit Line - SEL,8899.28,-9169.99,-270.71,27543.94,11180.90
63008 Amex Business Platinum CC - GG,228878.32,-218121.06,10757.26,349367.45,141642.94
7001 Chase Business Credit Line - GG,18716.42,-15607.34,3109.08,32798.52,13421.55
73003 Business Platinum CC - SEL,37809.46,-37711.65,97.81,307402.39,124782.70
AP Clearing Inventory Purchases,0.00,-19547.78,-19547.78,1979.00,766.70


In [50]:
!pwd

/home/jefferyp/Documents/projects/Cashflow_Forecasting


In [51]:
df_merge.to_csv('/home/jefferyp/Documents/projects/Cashflow_Forecasting/cashflow_2024.csv', index=False)